Let's connect with CIA Factbook Database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

Let's now view different tables in the connected database

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type ='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Let's feetch first few 5 rows from the `facts` table hosted by our database.

In [3]:
%%sql
SELECT * 	
FROM facts

LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Let's view min and max population and population growth.

In [4]:
%%sql
SELECT MIN(population) AS min_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population) AS max_pop,
       MAX(population_growth) AS max_pop_growth
FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,min_pop_growth,max_pop,max_pop_growth
0,0.0,7256490011,4.02


In [5]:
%%sql
SELECT name,population
FROM facts
WHERE population>=0
ORDER by population ASC
LIMIT 10;


 * sqlite:///factbook.db
Done.


name,population
Antarctica,0
Pitcairn Islands,48
Cocos (Keeling) Islands,596
Holy See (Vatican City),842
Niue,1190
Tokelau,1337
Christmas Island,1530
Svalbard,1872
Norfolk Island,2210
Falkland Islands (Islas Malvinas),3361


In [6]:
%%sql
SELECT name,population
FROM facts
ORDER by population DESC
LIMIT 10;


 * sqlite:///factbook.db
Done.


name,population
World,7256490011
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745


In [7]:
%%sql 
SELECT COUNT(*) AS Total_regions, (SELECT COUNT(*) FROM facts WHERE population>=0) AS non_null_regions
FROM facts;

 * sqlite:///factbook.db
Done.


Total_regions,non_null_regions
261,242


In [8]:
%%sql
SELECT Name, Population 
FROM facts 
WHERE population IS NULL

 * sqlite:///factbook.db
Done.


name,population
Ashmore and Cartier Islands,None
Coral Sea Islands,None
Heard Island and McDonald Islands,None
Clipperton Island,None
French Southern and Antarctic Lands,None
Bouvet Island,None
Jan Mayen,None
British Indian Ocean Territory,None
South Georgia and South Sandwich Islands,None
Navassa Island,None


Above outputs show that:
1. The database has a row that represents whole world (pop>7billion)
2. The database has a row that is for Antartica (pop=0)
3. The database has 19 regions with no value for population column

Let's now re-compute some summary statistics while ignoring World and Atartica from our analysis.

In [9]:
%%sql
SELECT name,
      population
FROM facts
WHERE Population >0
ORDER BY population 
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population
Pitcairn Islands,48
Cocos (Keeling) Islands,596
Holy See (Vatican City),842
Niue,1190
Tokelau,1337


In [10]:
%%sql
SELECT name,
      population_growth
FROM facts
WHERE population_growth >0
ORDER BY population_growth
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population_growth
Greece,0.01
Norfolk Island,0.01
Tokelau,0.01
Falkland Islands (Islas Malvinas),0.01
Guyana,0.02


In [11]:
%%sql
SELECT name,
      population
FROM facts
WHERE Population < (SELECT MAX(Population) FROM facts)
ORDER BY population DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674


In [12]:
%%sql
SELECT name,
      population_growth
FROM facts
WHERE population_growth >0
ORDER BY population_growth DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24


In [13]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts
WHERE name != 'World'

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


In [14]:
%%sql
SELECT name, ROUND(CAST(population AS float)/1000000,2)||'M' AS population
FROM facts
WHERE population > (SELECT AVG(population)
                    FROM facts
                    WHERE name != 'World') AND name!='World'

 * sqlite:///factbook.db
Done.


name,population
Afghanistan,32.56M
Algeria,39.54M
Argentina,43.43M
Bangladesh,168.96M
Brazil,204.26M
Burma,56.32M
Canada,35.1M
China,1367.49M
Colombia,46.74M
"Congo, Democratic Republic of the",79.38M


In [15]:
%%sql
SELECT Name, Area||' km²' AS area
FROM facts
WHERE Area < (SELECT AVG(Area)
                    FROM facts
                    WHERE name != 'World') AND name!='World'

 * sqlite:///factbook.db
Done.


name,area
Albania,28748 km²
Andorra,468 km²
Antigua and Barbuda,442 km²
Armenia,29743 km²
Austria,83871 km²
Azerbaijan,86600 km²
"Bahamas, The",13880 km²
Bahrain,760 km²
Bangladesh,148460 km²
Barbados,430 km²


Let's find densly populated countries!

In [16]:
%%sql 
SELECT Name, Population, Area, ROUND(CAST(Population as Float)/Area) AS pop_density
FROM facts
WHERE population > (SELECT AVG(population)
                    FROM facts
                    WHERE name != 'World') 
                    
                    AND
    
      area < (SELECT AVG(Area)
                    FROM facts
                    WHERE name != 'World')
ORDER BY pop_density DESC

 * sqlite:///factbook.db
Done.


name,population,area,pop_density
Bangladesh,168957745,148460,1138.0
"Korea, South",49115196,99720,493.0
Philippines,100998376,300000,337.0
Japan,126919659,377915,336.0
Vietnam,94348835,331210,285.0
United Kingdom,64088222,243610,263.0
Germany,80854408,357022,226.0
Italy,61855120,301340,205.0
Uganda,37101745,241038,154.0
Thailand,67976405,513120,132.0
